# Spark DFs

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark DataFrame").getOrCreate()


In [4]:
df = (spark.read
      .options(inferSchema=True, header=True)
      .csv("data/StudentData.csv"))
df.printSchema()


root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: integer (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



# Providing Schema

In [26]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),

    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("email", StringType(), True),
])


In [27]:
df = spark.read.options(header=True).schema(
    schema=schema).csv("data/StudentData.csv")
df.show()
df.printSchema()


+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

## RDD to DF

In [28]:
from pyspark import SparkConf, SparkContext
conf = SparkConf()
sc = SparkContext.getOrCreate("RDD to DF")

rdd = sc.textFile("data/StudentData.csv")
headers = rdd.first()
columns = headers.split(",")


rdd = rdd.filter(lambda x: x != headers).map(lambda x: x.split(','))

df = rdd.toDF(columns)
df.printSchema()

df = spark.createDataFrame(rdd, schema=schema)
df.printSchema()


root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



## Select DataFrame column

In [29]:
df = spark.read.options(header=True).schema(
    schema=schema).csv("data/StudentData.csv")

df.select("name", "gender").show(5)


+----------------+------+
|            name|gender|
+----------------+------+
| Hubert Oliveras|Female|
|Toshiko Hillyard|Female|
|  Celeste Lollis|  Male|
|    Elenore Choy|Female|
|  Sheryll Towler|  Male|
+----------------+------+
only showing top 5 rows



In [31]:
df.select("*").show()


+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 

## withColumn

In [37]:
from pyspark.sql.functions import col
df = spark.read.options(header=True).csv("data/StudentData.csv")
df = df.withColumn("roll", col("roll").cast("String"))
df.printSchema()


root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)



In [43]:
df.withColumn("normalized name", col("name").contains("a")).show()


+---+------+----------------+------+------+-----+--------------------+---------------+
|age|gender|            name|course|  roll|marks|               email|normalized name|
+---+------+----------------+------+------+-----+--------------------+---------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|           true|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|           true|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|          false|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|          false|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|          false|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|           true|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|           true|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|           true|
| 28|  Male|  Celeste Lollis|   MVC| 81492|

In [46]:
df.withColumnRenamed("name", "nombre").show(2)


+---+------+----------------+------+-----+-----+--------------------+
|age|gender|          nombre|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



In [50]:
df.select(col("name").alias("Nombre completo")).show()


+----------------+
| Nombre completo|
+----------------+
| Hubert Oliveras|
|Toshiko Hillyard|
|  Celeste Lollis|
|    Elenore Choy|
|  Sheryll Towler|
|  Margene Moores|
|     Neda Briski|
|    Claude Panos|
|  Celeste Lollis|
|  Cordie Harnois|
|       Kena Wild|
| Ernest Rossbach|
|  Latia Vanhoose|
|  Latia Vanhoose|
|     Neda Briski|
|  Latia Vanhoose|
|  Loris Crossett|
|  Annika Hoffman|
|   Santa Kerfien|
|Mickey Cortright|
+----------------+
only showing top 20 rows



# Filter / where

In [52]:
df.filter(col("course") == "DB").show()


+---+------+-----------------+------+-------+-----+--------------------+
|age|gender|             name|course|   roll|marks|               email|
+---+------+-----------------+------+-------+-----+--------------------+
| 28|Female|  Hubert Oliveras|    DB|  02984|   59|Annika Hoffman_Na...|
| 29|Female|     Elenore Choy|    DB|  32877|   29|Billi Clore_Mitzi...|
| 29|  Male|  Ernest Rossbach|    DB| 111449|   53|Maybell Duguay_Ab...|
| 28|Female|   Latia Vanhoose|    DB| 122502|   27|Latia Vanhoose_Mi...|
| 29|Female|   Latia Vanhoose|    DB| 152159|   27|Claude Panos_Sant...|
| 28|Female| Mickey Cortright|    DB| 192537|   62|Ernest Rossbach_M...|
| 28|Female|      Anna Santos|    DB| 311589|   79|Celeste Lollis_Mi...|
| 28|  Male|    Kizzy Brenner|    DB| 381712|   36|Paris Hutton_Kena...|
| 28|  Male| Toshiko Hillyard|    DB| 392218|   47|Leontine Phillips...|
| 29|  Male|     Paris Hutton|    DB| 481229|   57|Clementina Menke_...|
| 28|Female| Mickey Cortright|    DB| 551389|   43|

# Otras herramientas

## groupBy

In [57]:
df.groupBy("age").count().show()


+---+-----+
|age|count|
+---+-----+
| 29|  506|
| 28|  494|
+---+-----+



# distinct, sum, average, etc.

In [58]:
df.select("course").distinct().show()


+------+
|course|
+------+
|    PF|
|    DB|
|   MVC|
|   DSA|
| Cloud|
|   OOP|
+------+



In [60]:
df.groupBy("course", "gender").count().orderBy("course").show()


+------+------+-----+
|course|gender|count|
+------+------+-----+
| Cloud|Female|  106|
| Cloud|  Male|   86|
|    DB|  Male|   82|
|    DB|Female|   75|
|   DSA|Female|   98|
|   DSA|  Male|   78|
|   MVC|  Male|   86|
|   MVC|Female|   71|
|   OOP|Female|   82|
|   OOP|  Male|   70|
|    PF|  Male|   97|
|    PF|Female|   69|
+------+------+-----+



In [61]:
from pyspark.sql.functions import count, avg, min, mean, sum


In [72]:
df.groupBy("course", "gender").agg(count("*"), sum("marks"),
                                   avg(col("marks")).alias("average")).orderBy("course").show()


+------+------+--------+----------+------------------+
|course|gender|count(1)|sum(marks)|           average|
+------+------+--------+----------+------------------+
| Cloud|  Male|      86|    5127.0|59.616279069767444|
| Cloud|Female|     106|    6316.0| 59.58490566037736|
|    DB|  Male|      82|    5073.0| 61.86585365853659|
|    DB|Female|      75|    4197.0|             55.96|
|   DSA|Female|      98|    6124.0| 62.48979591836735|
|   DSA|  Male|      78|    4826.0| 61.87179487179487|
|   MVC|  Male|      86|    5241.0| 60.94186046511628|
|   MVC|Female|      71|    4344.0|61.183098591549296|
|   OOP|Female|      82|    4682.0| 57.09756097560975|
|   OOP|  Male|      70|    4234.0| 60.48571428571429|
|    PF|Female|      69|    3973.0| 57.57971014492754|
|    PF|  Male|      97|    5960.0| 61.44329896907217|
+------+------+--------+----------+------------------+



## groupBy and Filtering

In [74]:
df.filter(df.gender == "Male").groupBy("course").count().show()


+------+-----+
|course|count|
+------+-----+
|    PF|   97|
|    DB|   82|
|   MVC|   86|
|   DSA|   78|
| Cloud|   86|
|   OOP|   70|
+------+-----+



In [75]:
df.createOrReplaceTempView("Student")

In [78]:
spark.sql("select * from Student").show()

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...|
| 29|Female|       Kena Wild|   DSA|102285|   35|Dustin Feagins_Ma...|
| 29| 